In [1]:
MODELS_PATH = '../model/'
!ls -1 $MODELS_PATH

top.ev
top.ma


In [54]:
def replaceParams (content, params):
    new_content = content
    for param, value in params.items():
        search = r'{} : .*'.format(param)
        replace = r'{} : {}'.format(param, value)
        new_content = re.sub(search, replace, new_content, flags = re.M)
    return new_content

def run_simulation(sim_path, params, top_model_content, simulation_args):
    # acomodo paths dentro de la carpeta de la simulacion
    time = simulation_args['time']
    logs_path = sim_path + simulation_args['logs_rel_path']
    output_path = sim_path + simulation_args['output_rel_path'] + 'output'
    stdout_path = sim_path + simulation_args['stdout_rel_path'] + 'stdout'
    top_model_path = sim_path + simulation_args['model_rel_path']

    # creo carpeta de simulacion
    ! mkdir {sim_path} {logs_path}
    
    # genero top_model con los params reemplazados
    new_top_model_content = replaceParams(top_model_content, params)
    with open (top_model_path, 'a' ) as f:
        f.write(new_top_model_content)
    
    # ejecuto simu
    !../src/bin/cd++ -m {top_model_path} -t {time} -l {logs_path} -o {output_path} > {stdout_path}  

In [59]:
import re

# abro el top model
with open (MODELS_PATH + 'top.ma', 'r' ) as f:
    top_model_content = f.read()

# borro las simulaciones previamente corridas    
!ls | grep sim | xargs rm -rf

# corro nuevas simulaciones
for consumption_factor in [0.1, 0.2]:
    for conversion_factor in [0.4, 0.5]:
        sim_path = 'sim-cons-{}-conv-{}/'.format(consumption_factor, conversion_factor)

        params_to_replace = {'consumption_factor': consumption_factor,
                             'conversion_factor': conversion_factor}
        simulation_args = {'time': '00:05:000:0',
                           'model_rel_path': 'top.ma', 
                           'logs_rel_path': 'logs/',
                           'output_rel_path': './',
                           'stdout_rel_path': './'}
        print ("{} Running simulation".format(params_to_replace))
        run_simulation(sim_path, params_to_replace, top_model_content, simulation_args)



{'consumption_factor': 0.1, 'conversion_factor': 0.4} Running simulation
{'consumption_factor': 0.1, 'conversion_factor': 0.5} Running simulation
{'consumption_factor': 0.2, 'conversion_factor': 0.4} Running simulation
{'consumption_factor': 0.2, 'conversion_factor': 0.5} Running simulation
